Install Langchain Python Package and Python 3.10+

In [5]:
from langchain.embeddings import GPT4AllEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


# Import PyPDF 
import pypdf

# Import Gradio Tool
import gradio as gr

import os

In [3]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

In [11]:
llm = ChatOpenAI(temperature = 0.7)

In [3]:
def email_temp(template_path = ""):
    # Define prompt
    prompt_text = """You are a project manager who is responsible for replying to emails from customers who are looking to rent cranes from our company, ConstructionCo. You need to draft an email template response for us to send out based on the received email. We want to reply positively to the email and confirm the dates and locations to the customer. You MUST keep the email brief and to the point. The recieved email is below:
    {email_template} """

    prompt = PromptTemplate.from_template(prompt_text)

    # Define LLM chain
    llm = ChatOpenAI(temperature=0, engine="gpt-4-32k")
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    pdf_reader = PdfReader(template_path)
    pdf_text = ""
    for i, page in enumerate(pdf_reader.pages):
        # print(page.extract_text())
        pdf_text += page.extract_text() + "\n"

    return llm_chain.run(pdf_text)

In [8]:
loader = PyPDFLoader("./2023_ASQ_PRQC_Presentation/nasa_fmea_guidance.pdf")
pages = loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)

In [9]:
vectordb = FAISS.from_documents(all_splits, embedding=GPT4AllEmbeddings())

In [12]:
def question2pdf(message, history):
    qachain=RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())
    return qachain({"query": message})['result']

app = gr.ChatInterface(question2pdf)

app.launch(share=False)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://11b64aad19f6f3f2be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [13]:
app.close()

Closing server running on port: 7863
